In [1]:
!pip3 install submodules/diff-gaussian-rasterization submodules/simple-knn plyfile pyproj
!apt update && apt install ffmpeg -y 
!pip3 install ffmpeg-python
!pip3 install pycolmap 

Processing ./submodules/diff-gaussian-rasterization
  Preparing metadata (setup.py) ... done
Processing ./submodules/simple-knn
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 47.1 MB/s eta 0:00:00
  Created wheel for diff-gaussian-rasterization: filename=diff_gaussian_rasterization-0.0.0-cp310-cp310-linux_x86_64.whl size=2951764 sha256=1ab8ec05c5348e91eee53d425db7a187ca7f48278b688660214be1566939e8e2
  Stored in directory: /root/.cache/pip/wheels/86/3e/74/8450af5b4c7bb21b5697f5383860156f6e5b0690f5dd2695f7
  Created wheel for simple-knn: filename=simple_knn-0.0.0-cp310-cp310-linux_x86_64.whl size=2870138 sha256=61185cbd944ba8971c8961e8093bf7562f85b28c0bc3d439c4a8594628317e29
  Stored in directory: /root/.cache/pip/wheels/68/4d/2a/05a0b1d5602af67a51461e8681d3f756c354a44b47c099782b
Successfully built diff-gaussian-rasterization simple-knn
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1581 B]
Get

### 0 動画から点群を推定

In [1]:
import os 
import ffmpeg
import os
import glob
root = "/work/data/headphone/"
# 入力動画ファイルのパス
input_video = "IMG_4830.MOV"
video_path = os.path.join(root,input_video)

print(video_path)
# 出力フォルダのパス
output_folder = os.path.join(root,"input")

# 出力フォルダが存在しない場合は作成
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 動画情報の取得
probe = ffmpeg.probe(video_path)
video_info = next(s for s in probe['streams'] if s['codec_type'] == 'video')
width = int(video_info['width'])
height = int(video_info['height'])

# フレーム抽出（1秒間に10フレーム）
(
    ffmpeg
    .input(video_path)
    .filter('fps', fps=10)  # ここを10 fpsに変更
    .output(f'{output_folder}/frame_%05d.png', start_number=0)
    .overwrite_output()
    .run(capture_stdout=True, capture_stderr=True)
)

print("フレーム抽出が完了しました。")
frame_count = len(glob.glob(os.path.join(output_folder, "frame_*.png")))
print(f"抽出されたフレーム数: {frame_count}")


/work/data/headphone/IMG_4830.MOV
フレーム抽出が完了しました。
抽出されたフレーム数: 192


In [2]:
import os
import logging
from argparse import ArgumentParser
import shutil
import pycolmap
import sqlite3
import warnings
src_folder = output_folder

image_dir = output_folder
os.makedirs(os.path.join(root + "distorted/sparse"), exist_ok=True)
db_path = os.path.join(root,"distorted/database.db")
# print(db_path)
sift_options = pycolmap.SiftExtractionOptions()
pycolmap.extract_features(
    database_path=db_path, 
    image_path=image_dir,
    sift_options=sift_options,
)



W20241001 04:11:18.647970 140225580666880 feature_extraction.cc:406] Your current options use the maximum number of threads on the machine to extract features. Extracting SIFT features on the CPU can consume a lot of RAM per thread for large images. Consider reducing the maximum image size and/or the first octave or manually limit the number of extraction threads. Ignore this warning, if your machine has sufficient memory for the current settings.
I20241001 04:11:18.660764 140224610416192 misc.cc:198] 
Feature extraction
I20241001 04:11:18.669064 140224483018304 sift.cc:722] Creating SIFT CPU feature extractor
I20241001 04:11:18.669081 140224474625600 sift.cc:722] Creating SIFT CPU feature extractor
I20241001 04:11:18.669371 140224491411008 sift.cc:722] Creating SIFT CPU feature extractor
I20241001 04:11:18.670371 140224499803712 sift.cc:722] Creating SIFT CPU feature extractor
I20241001 04:11:18.672758 140224466232896 feature_extraction.cc:257] Processed file [1/192]
I20241001 04:11:1

OperationalError: no such column: name

In [13]:
# 特徴点の情報を表示する
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT rows FROM 'keypoints'")
keypoint_counts = cursor.fetchall()
print("画像ごとの特徴点数:")
for  n ,count in enumerate(keypoint_counts):
    print(f"{n}frame {count[0]}個の特徴点")
total_keypoints = sum(count[0] for count in keypoint_counts)
print(f"\n合計特徴点数: {total_keypoints}個")
conn.close()

画像ごとの特徴点数:
0frame 973個の特徴点
1frame 1182個の特徴点
2frame 1352個の特徴点
3frame 1506個の特徴点
4frame 694個の特徴点
5frame 781個の特徴点
6frame 745個の特徴点
7frame 929個の特徴点
8frame 872個の特徴点
9frame 897個の特徴点
10frame 744個の特徴点
11frame 1087個の特徴点
12frame 968個の特徴点
13frame 1124個の特徴点
14frame 742個の特徴点
15frame 981個の特徴点
16frame 1222個の特徴点
17frame 1105個の特徴点
18frame 796個の特徴点
19frame 651個の特徴点
20frame 899個の特徴点
21frame 952個の特徴点
22frame 1241個の特徴点
23frame 1130個の特徴点
24frame 1181個の特徴点
25frame 1227個の特徴点
26frame 543個の特徴点
27frame 428個の特徴点
28frame 902個の特徴点
29frame 885個の特徴点
30frame 922個の特徴点
31frame 757個の特徴点
32frame 671個の特徴点
33frame 407個の特徴点
34frame 584個の特徴点
35frame 234個の特徴点
36frame 358個の特徴点
37frame 200個の特徴点
38frame 371個の特徴点
39frame 215個の特徴点
40frame 142個の特徴点
41frame 131個の特徴点
42frame 157個の特徴点
43frame 36個の特徴点
44frame 40個の特徴点
45frame 61個の特徴点
46frame 14個の特徴点
47frame 60個の特徴点
48frame 69個の特徴点
49frame 71個の特徴点
50frame 33個の特徴点
51frame 59個の特徴点
52frame 134個の特徴点
53frame 72個の特徴点
54frame 200個の特徴点
55frame 213個の特徴点
56frame 237個の特徴点
57frame 295個の特徴点
58frame 316個

In [14]:
# Future_matching
pycolmap.match_exhaustive(db_path)



I20241001 04:38:46.933023 140224610416192 misc.cc:198] 
Feature matching
I20241001 04:38:46.941031 140224596780608 sift.cc:1457] Creating SIFT CPU feature matcher
I20241001 04:38:46.944455 140224474625600 sift.cc:1457] Creating SIFT CPU feature matcher
I20241001 04:38:46.944464 140224483018304 sift.cc:1457] Creating SIFT CPU feature matcher
I20241001 04:38:46.944529 140224466232896 sift.cc:1457] Creating SIFT CPU feature matcher
I20241001 04:38:47.055229 140224610416192 pairing.cc:168] Generating exhaustive image pairs...
I20241001 04:38:47.055816 140224610416192 pairing.cc:202] Matching block [1/4, 1/4]
I20241001 04:39:15.569132 140224610416192 feature_matching.cc:46]  in 28.513s
I20241001 04:39:15.570700 140224610416192 pairing.cc:202] Matching block [1/4, 2/4]
I20241001 04:39:56.499344 140224610416192 feature_matching.cc:46]  in 40.929s
I20241001 04:39:56.509945 140224610416192 pairing.cc:202] Matching block [1/4, 3/4]
I20241001 04:40:29.152666 140224610416192 feature_matching.cc:46

In [ ]:
# マッチングした数を取得
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM matches")
total_matches = cursor.fetchone()[0]
cursor.execute("SELECT pair_id, rows FROM matches")
pair_matches = cursor.fetchall()
print(f"総マッチング数: {total_matches}")
print("\n画像ペアごとのマッチング数:")
for pair_id, count in pair_matches:
    image_id1 = pair_id >> 32
    image_id2 = pair_id & 0xFFFFFFFF
    if count > 0:
        print(f"画像 {image_id1} と 画像 {image_id2}: {count}個のマッチ")
avg_matches = total_matches / len(pair_matches) if pair_matches else 0
print(f"\n平均マッチング数: {avg_matches:.2f}")
conn.close()

In [6]:
# Mapping
# KernelRestartなどで初期値が飛んだとき用
# root = "/work/data/headphone/" 
# import os
# import logging
# from argparse import ArgumentParser
# import shutil
# import pycolmap
# import sqlite3
# import warnings
# output_path = os.path.join(root , "distorted/sparse")
# db_path = os.path.join(root,"distorted/database.db")
# image_dir = os.path.join(root,"input")
# print(output_path)
increment_options = pycolmap.IncrementalPipelineOptions()
increment_options.ba_global_function_tolerance=0.000001
# print(help(increment_options))
maps = pycolmap.incremental_mapping(db_path,image_dir,output_path,increment_options)
maps[0].write(output_path)

I20241001 05:02:51.570674 139639277850624 incremental_mapper.cc:225] Loading database
I20241001 05:02:51.637797 139639277850624 database_cache.cc:65] Loading cameras...
I20241001 05:02:51.639814 139639277850624 database_cache.cc:75]  198 in 0.002s
I20241001 05:02:51.639832 139639277850624 database_cache.cc:83] Loading matches...
I20241001 05:02:51.744591 139639277850624 database_cache.cc:89]  1879 in 0.105s
I20241001 05:02:51.744629 139639277850624 database_cache.cc:105] Loading images...
I20241001 05:02:51.938539 139639277850624 database_cache.cc:155]  192 in 0.194s (connected 182)
I20241001 05:02:51.938580 139639277850624 database_cache.cc:166] Building correspondence graph...
I20241001 05:02:51.953741 139639277850624 database_cache.cc:195]  in 0.015s (ignored 0)
I20241001 05:02:51.954078 139639277850624 timer.cc:91] Elapsed time: 0.005 [minutes]
I20241001 05:02:51.959029 139639277850624 incremental_mapper.cc:265] Finding good initial image pair
I20241001 05:02:52.146685 139639277850

In [10]:
# Image Undest
mvs_path = os.path.join(root,"distorted/sparse/0")
output_path =os.path.join(root) 
db_path = os.path.join(root,"distorted/database.db")
pycolmap.undistort_images(output_path, mvs_path,image_dir,output_type="COLMAP")
files = os.listdir(root + "/sparse")


I20241001 05:23:48.413034 139639277850624 images.cc:103]  => Reconstruction with 174 images and 9231 points
I20241001 05:23:48.414487 139639277850624 misc.cc:198] 
Image undistortion
I20241001 05:23:48.417533 139639277850624 undistortion.cc:214] Undistorting image [1/174]
I20241001 05:23:49.425344 139639277850624 undistortion.cc:214] Undistorting image [2/174]
I20241001 05:23:51.290029 139639277850624 undistortion.cc:214] Undistorting image [3/174]
I20241001 05:23:51.290062 139639277850624 undistortion.cc:214] Undistorting image [4/174]
I20241001 05:23:51.290065 139639277850624 undistortion.cc:214] Undistorting image [5/174]
I20241001 05:23:51.978611 139639277850624 undistortion.cc:214] Undistorting image [6/174]
I20241001 05:23:52.187010 139639277850624 undistortion.cc:214] Undistorting image [7/174]
I20241001 05:23:52.650352 139639277850624 undistortion.cc:214] Undistorting image [8/174]
I20241001 05:23:53.305981 139639277850624 undistortion.cc:214] Undistorting image [9/174]
I202410

In [ ]:
# file Reformat
os.makedirs(root + "/sparse/0", exist_ok=True)
# Copy each file from the source directory to the destination directory
for file in files:
    if file == '0':
        continue
    source_file = os.path.join(root, "sparse", file)
    destination_file = os.path.join(root, "sparse", "0", file)
    shutil.move(source_file, destination_file)